In [1]:
from docplex.mp.model import Model

In [3]:
items = range(5)
item_sizes = [20, 45, 50, 55, 75]
item_demands = [48, 35, 24, 10, 8]
roll_width = 110
EPS = 0.0001

In [4]:
max_cut = 100
cuts = range(max_cut)
mdl = Model('Custstock')
cut_vars = mdl.integer_var_matrix(cuts, items, lb=0, ub=10, name="cuts")
used_vars = mdl.binary_var_dict(cuts, name="used")
# Objective
cost = mdl.sum(used_vars[c] for c in cuts)
obj = mdl.minimize(cost)
# link used and cuts
mdl.add_constraints(mdl.sum(cut_vars[c, i]
                    for i in items) <= 100*used_vars[c] for c in cuts)
# Cover demand
mdl.add_constraints(mdl.sum(cut_vars[c, i]
                    for c in cuts) >= item_demands[i] for i in items)
# Roll width
mdl.add_constraints(mdl.sum(item_sizes[i] * cut_vars[c, i]
                    for i in items) <= roll_width for c in cuts)
mdl.print_information()
mdl.solve(log_output=True)
mdl.report()


Model: Custstock
 - number of variables: 600
   - binary=100, integer=500, continuous=0
 - number of constraints: 205
   - linear=205
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve modified 400 coefficients.
Reduced MIP has 205 rows, 600 columns, and 1600 nonzeros.
Reduced MIP has 200 binaries, 400 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.85 ticks)
Found incumbent of value 100.000000 after 0.03 sec. (3.38 ticks)
Probing time = 0.00 sec. (0.13 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 205 rows, 600 columns, and 1600 nonzeros.
Reduced MIP has 200 binaries, 400 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.93 ticks)
Probing time = 0.00 sec. (0.13 ticks)
Clique table members: 100.
MIP emphasis: balance optimality and feasibility.
MIP search method: dyna

In [8]:
master_mdl = Model('Master-Cutstock')
dummy_vars = master_mdl.continuous_var_dict(
    items, lb=0, ub=max_cut, name="dummy_cut")
cost = master_mdl.sum(1000*dummy_vars[i] for i in items)
obj = master_mdl.minimize(cost)
cts = master_mdl.add_constraints(
    dummy_vars[i] >= item_demands[i] for i in items)
master_mdl.solve()
master_mdl.report()


* model Master-Cutstock solved with objective = 125000.000


In [12]:
duals = master_mdl.dual_values(cts)

In [13]:
sub_mdl = Model('Sub-Cutstock')
item_vars = sub_mdl.integer_var_dict(items, ub=999999, name='items')
sub_mdl.add_constraint(sub_mdl.sum(
    item_sizes[i]*item_vars[i] for i in items) <= roll_width)
sub_mdl.minimize(1 - sub_mdl.sum(duals[i]*item_vars[i] for i in items))
sub_mdl.solve()
sub_mdl.report()


* model Sub-Cutstock solved with objective = -4999.000


In [14]:
if sub_mdl.objective_value < -EPS:
    new_pattern = [item_vars[i].solution_value for i in items]
    print(f"New pattern {new_pattern}")
    patterns.append(new_pattern)

    new_var = master_mdl.continuous_var(
        lb=0, ub=max_cut, name="cut_{0}".format(len(patterns)))
    cut_vars.append(new_var)

    cost += new_var
    for i in items:
        cts[i].lhs += new_pattern[i] * new_var
    master_mdl.solve()
    master_mdl.report()
    duals = master_mdl.dual_values(cts)
else:
    break


{0: docplex.mp.Var(type=I,name='items_0',ub=999999),
 1: docplex.mp.Var(type=I,name='items_1',ub=999999),
 2: docplex.mp.Var(type=I,name='items_2',ub=999999),
 3: docplex.mp.Var(type=I,name='items_3',ub=999999),
 4: docplex.mp.Var(type=I,name='items_4',ub=999999)}